# Module 1: ROI Definition and Imagery Acquisition
Define the region of interest (ROI), setup data access to Copernicus, and download the relevant imagery.

1. SentinelHub must be installed: https://sentinelhub-py.readthedocs.io/en/latest/install.html
2. Access must be provided to the Copernicus Data Space Ecosystem (CDSE): https://documentation.dataspace.copernicus.eu/APIs/SentinelHub/Overview/Authentication.html

See Sentinel Hub Process API instructions for more details (https://sentinelhub-py.readthedocs.io/en/latest/examples/process_request_cdse.html#Credentials).v>

Package installation requirements:
- Matplotlib (`matplotlib`): https://pypi.org/project/matplotlib/
- NumPy (`numpy`): https://pypi.org/project/numpy/
- SentinelHub (`sentinelhub`): https://pypi.org/project/sentinelhub/

***
## User Input
Define the key inputs for the module here:
1. Region of interest (ROI) by defining minimum/maximum decimal latitude/longitude.
2. Start and end time in the format: yyyy-mm-dd
3. Data output location.
4. Data access credentials.

### Spatial/Temporal Extent

In [ ]:
# decimal latitude and longitude extents
# accra example
latmin = 5.4219
latmax = 5.5504
lonmin = -0.3547
lonmax = -0.1933

# start and end times in yyyy-mm-dd format
start_date = "2018-10-31" 
end_date = "2018-11-01"

### Data Output Location

In [ ]:
# define output location for the generated .tiff
output_folder = ''
output_name = 'test' #output file name

### Define CDSE Credentials
The **client ID** and **client secret** must be defined here to allow acces to the CDSE database. To do this, an OAuth client must be registered via the Sentinel Hub services dashboard (https://shapps.dataspace.copernicus.eu/dashboard/#/) to grant remote access to Copernicus. <br><br>For full instructions, visit: https://documentation.dataspace.copernicus.eu/APIs/SentinelHub/Overview/Authentication.html.
<br><br>
Provide the **client ID** and **client secret** as above. The **instance ID** is a string to describe the name of the authentication instance, which can be any relatively short string. The authentication profile can also be given a name to save for use later.

In [ ]:
client_id = ''
client_secret = ''
instance_id = 'sentinelhub_cid'
profile_name = 'test-profile'

***

## Load Packages and Define Functions

In [ ]:
from sentinelhub import WebFeatureService
from sentinelhub import (
    CRS, 
    BBox, 
    DataCollection, 
    SHConfig, 
    DownloadRequest,
    MimeType, 
    MosaickingOrder, 
    SentinelHubDownloadClient,
    SentinelHubRequest,
    bbox_to_dimensions
)
import matplotlib.pyplot as plt
import numpy as np
from typing import Any
import os

In [ ]:
# define image plotting function to display acquired imagery (from https://sentinelhub-py.readthedocs.io/en/latest/index.html)
def plot_image(
    image: np.ndarray, factor: float = 1.0, clip_range: tuple[float, float] | None = None, **kwargs: Any
) -> None:
    _, ax = plt.subplots(nrows=1, ncols=1, figsize=(15, 15))
    if clip_range is not None:
        ax.imshow(np.clip(image * factor, *clip_range), **kwargs)
    else:
        ax.imshow(image * factor, **kwargs)
    ax.set_xticks([])
    ax.set_yticks([])

## Construct Config File

In [ ]:
# define Copernicus Data Space Ecosystem (CDSE) profile
config = SHConfig()
config.sh_client_id = client_id
config.sh_client_secret = client_secret
config.sh_base_url = 'https://sh.dataspace.copernicus.eu'
config.sh_token_url = 'https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token'
config.instance_id = instance_id
config.save(profile_name)

SHConfig.get_config_location()

print(config)

## Setup ROI and Time Range
The ROI is defined using a bounding box, providing min/max extent for latitude and longitude as in the **User Input** section for Module 1.

In [ ]:
req_coords_wgs84 = (lonmin, latmin, lonmax, latmax)
resolution = 10
req_bbox = BBox(bbox=req_coords_wgs84, crs=CRS.WGS84)
req_size = bbox_to_dimensions(req_bbox, resolution=resolution)

print(f"Image shape at {resolution} m resolution: {req_size} pixels")

## Download and Display True Colour Image at ROI
The true colour image that covers the ROI is displayed here, for Sentinel-2 level 2A data.

In [ ]:
# image acquisition script (from https://sentinelhub-py.readthedocs.io/en/latest/examples/process_request_cdse.html#Credentials)
evalscript_true_color = """
    //VERSION=3

    function setup() {
        return {
            input: [{
                bands: ["B02", "B03", "B04"]
            }],
            output: {
                bands: 3
            }
        };
    }

    function evaluatePixel(sample) {
        return [sample.B04, sample.B03, sample.B02];
    }
"""

request_true_color = SentinelHubRequest(
    evalscript = evalscript_true_color,
    input_data = [
        SentinelHubRequest.input_data(
            DataCollection.SENTINEL2_L2A.define_from("s2l2a", service_url=config.sh_base_url),
            time_interval=(start_date, end_date),
        )
    ],
    responses = [SentinelHubRequest.output_response("default", MimeType.PNG)],
    bbox = req_bbox,
    size = req_size,
    config = config,
)

In [ ]:
%%time
# request true colour image
true_color_imgs = request_true_color.get_data()

In [ ]:
image = true_color_imgs[0]

# factor 1/255 to scale between 0-1
# factor 3.5 to increase brightness
plot_image(image, factor=3.5 / 255, clip_range=(0, 1))

## Download and Display False Colour Image at ROI
The false colour image that covers the ROI is displayed here, for Sentinel-2 level 2A data. The image is a mosaic of images collected over the specified date range.

In [ ]:
# image acquisition script (from https://sentinelhub-py.readthedocs.io/en/latest/examples/process_request_cdse.html#Credentials)
evalscript_all_bands = """
    //VERSION=3
    function setup() {
        return {
            input: [{
                bands: ["B01","B02","B03","B04","B05","B06","B07","B08","B8A","B09","B11","B12","SCL"],
                units: "DN"
            }],
            output: {
                bands: 13,
                sampleType: "INT16"
            }
        };
    }

    function evaluatePixel(sample) {
        return [sample.B01,
                sample.B02,
                sample.B03,
                sample.B04,
                sample.B05,
                sample.B06,
                sample.B07,
                sample.B08,
                sample.B8A,
                sample.B09,
                sample.B11,
                sample.B12,
                sample.SCL];
    }
"""

request_all_bands = SentinelHubRequest(
    data_folder = output_folder,
    evalscript = evalscript_all_bands,
    input_data = [
        SentinelHubRequest.input_data(
            data_collection=DataCollection.SENTINEL2_L2A.define_from("s2l2a", service_url=config.sh_base_url),
            time_interval=(start_date, end_date),
            mosaicking_order=MosaickingOrder.LEAST_CC,
        )
    ],
    responses = [SentinelHubRequest.output_response("default", MimeType.TIFF)],
    bbox = req_bbox,
    size = req_size,
    config = config,
)

In [ ]:
%%time
# request imagery with all bands
all_bands_response = request_all_bands.get_data()

In [ ]:
# plot false colour image using bands B03, B04, and B08
plot_image(all_bands_response[0][:, :, [2, 3, 7]], factor=3.5/1e4, clip_range=(0, 1))

## Save Dataset
Save the multispectral Sentinel-2 data as a GeoTIFF.

In [ ]:
%%time
request_all_bands.save_data()

for folder, _, filenames in os.walk(request_all_bands.data_folder):
    for filename in filenames:
        f = os.path.join(folder, filename)
        comp = f.split('\\')
        req_id = comp[-2] # request ID that is automatically generated

print("Generated request ID: "+req_id) # the request ID can be used to continue into Module 2